**Выполнять в среде moldyn**

https://docs.mdanalysis.org/1.0.0/documentation_pages/analysis/contacts.html

In [21]:
import MDAnalysis
from MDAnalysis.analysis import contacts
import nglview as nv
import pandas as pd

In [22]:
import pynucl
import pandas as pd
import numpy as np
from pynucl.entity_detector import detect_entities_independent
from pynucl.seq_utils import residues_3to1
from multiprocessing import Pool
from tqdm.auto import tqdm

In [23]:

from scipy.spatial import KDTree
from scipy.spatial import cKDTree
import numpy as np
from collections import OrderedDict

# Разбор кусков кода

Загружаем структуру

In [4]:

!wget https://files.rcsb.org/view/1KX5.pdb

--2022-06-22 20:28:43--  https://files.rcsb.org/view/1KX5.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.49
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘1KX5.pdb.2’

1KX5.pdb.2              [    <=>             ]   1.40M  1.61MB/s    in 0.9s    

2022-06-22 20:28:45 (1.61 MB/s) - ‘1KX5.pdb.2’ saved [1464723]



MDAnalysis begins with Universe, which contains all the information describing a molecular dynamics system.  
https://userguide.mdanalysis.org/stable/universe.html

In [5]:
u=MDAnalysis.Universe('1KX5.pdb')

/opt/miniconda3/envs/moldyn/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [6]:
s=u.select_atoms('all')

In [9]:
nv.show_mdanalysis(s)

NGLWidget()

In [10]:
group1 = u.select_atoms('protein')
group2 = u.select_atoms('protein')

In [11]:
group1.names

array(['N', 'CA', 'C', ..., 'CE', 'NZ', 'OXT'], dtype=object)

In [12]:
dist = contacts.distance_array(group1.positions, group2.positions)

In [13]:
dist.shape

(7586, 7586)

https://github.com/intbio/pymolint/blob/28bb9f7d633d9ff78e55b536a8d6089096ef96f4/pymolint/base.py

In [14]:
# https://github.com/intbio/pymolint/blob/28bb9f7d633d9ff78e55b536a8d6089096ef96f4/pymolint/base.py

def find_contacts(A_coordinates,A_indices,B_coordinates,B_indices,d_threshold=3.9,exclude_bonded=False,half_matrix=True):
	""" Function finds contacts between atoms in groups A and B.
	If exclude_bonded is set to True, we assume that set A = set B,
	and we need to exclude bonded neighbors up to three bonds away (inclusive: 1-4 interactions are also excluded).
	Self-interaction has also to be excluded.
    and only half-matrix is output by default (change parameter to override)
	the bonded threshold is set to colser than 1.7 A 
	Atoms must be heavy!
	Parameters
	----------
	A_coordinates - list of tuples with (x,y,z) of group A.
	A_indices - list of indices of atoms corresponding to the list of coordinates. This facilitates using and avoids unneeded conversion by user.
	B_coordinates - same as for A.
	B_indices - same as for A.
	d_threshold - distance threshold for contacts
	returns - list of tuples (A_ind, B_ind, distance)
	"""
	bond_threshold=1.7

	A_tree=cKDTree(A_coordinates)
	B_tree=cKDTree(B_coordinates)

	# print A_indices
	# print B_indices
	#We can calculate neighbors but it is not needed since it is automatically done by distance matrix calculation
	#neighbors=A_tree.query_ball_tree(B_tree,d_threshold)
	#print neighbors #it is a list of lists

	dok_dist=A_tree.sparse_distance_matrix(B_tree,d_threshold)
	# Now we have a sparse distance matrix in a dictionary of keys format (see scipy.sparse.dok_matrix)

	# In case we need to exclude bonded let's do it
	if(exclude_bonded):
		bonded=A_tree.query_pairs(bond_threshold)
		bonded_upto_1_4=bonded_1_4(bonded)
		# print bonded_upto_1_4
		#Bonded is a set
		#Now we need also to construct 1-3, 1-4 bonded sets

	#Now let's iterate through the matrix and form an output list of tuples

	contact_list=list()
	# print dok_dist
	for key in iter(dok_dist.keys()):
		if(exclude_bonded):
			if((key not in bonded_upto_1_4) and ((key[1],key[0]) not in bonded_upto_1_4)):
				if(dok_dist.get(key)>0.1):
					# print dok_dist.get(key) # self-exclusion
					if(half_matrix):
						if(key[0]<=key[1]): #only half of matrix is needed
							contact_list.append((A_indices[key[0]],B_indices[key[1]],dok_dist.get(key)))
					else:
						contact_list.append((A_indices[key[0]],B_indices[key[1]],dok_dist.get(key)))

			# else:
				# print "Excluding bonded"
		else:
			contact_list.append((A_indices[key[0]],B_indices[key[1]],dok_dist.get(key)))
	


	return contact_list

In [15]:
dist = find_contacts(group1.positions,group1.ids,group2.positions,group2.ids,d_threshold=3.9,exclude_bonded=False,half_matrix=True) #[:10]

In [76]:
len(dist)

91446

In [77]:
dist[:20]

[(6024, 6024, 0.0),
 (6025, 6024, 1.4884386629602506),
 (6026, 6024, 2.4783904553732072),
 (6027, 6024, 3.301415790537391),
 (6028, 6024, 2.467477899293438),
 (6029, 6024, 3.0969184925020836),
 (6035, 6024, 3.3105519240815426),
 (6036, 6024, 3.8654675659396913),
 (6024, 6025, 1.4884386629602506),
 (6025, 6025, 0.0),
 (6026, 6025, 1.5245769153993387),
 (6027, 6025, 2.398004488775112),
 (6028, 6025, 1.5278576556998114),
 (6029, 6025, 2.4287079566068535),
 (6030, 6025, 3.8055596163080807),
 (6035, 6025, 3.7877813938954006),
 (6024, 6026, 2.4783904553732072),
 (6025, 6026, 1.5245769153993387),
 (6026, 6026, 0.0),
 (6027, 6026, 1.231346131322434)]

In [78]:
dist = [tup for tup in dist if tup[2] != 0]
dist[:20]

[(6025, 6024, 1.4884386629602506),
 (6026, 6024, 2.4783904553732072),
 (6027, 6024, 3.301415790537391),
 (6028, 6024, 2.467477899293438),
 (6029, 6024, 3.0969184925020836),
 (6035, 6024, 3.3105519240815426),
 (6036, 6024, 3.8654675659396913),
 (6024, 6025, 1.4884386629602506),
 (6026, 6025, 1.5245769153993387),
 (6027, 6025, 2.398004488775112),
 (6028, 6025, 1.5278576556998114),
 (6029, 6025, 2.4287079566068535),
 (6030, 6025, 3.8055596163080807),
 (6035, 6025, 3.7877813938954006),
 (6024, 6026, 2.4783904553732072),
 (6025, 6026, 1.5245769153993387),
 (6027, 6026, 1.231346131322434),
 (6028, 6026, 2.500992263099066),
 (6029, 6026, 1.3291396494441028),
 (6030, 6026, 2.4331610025109307)]

In [79]:
id2name={}
id2segid={}
id2resid={}                                                            
              
id2resid={a.id:a.resid for a in group1.atoms}

        
id2resname={a.id:a.resname for a in group1.atoms}
                                                         
id2segid={a.id:a.segid for a in group1.atoms}
                                                                    
id2name={a.id:a.name for a in group1.atoms}


https://github.com/intbio/pymolint/blob/28bb9f7d633d9ff78e55b536a8d6089096ef96f4/pymolint/mol_int.py

In [80]:
# https://github.com/intbio/pymolint/blob/28bb9f7d633d9ff78e55b536a8d6089096ef96f4/pymolint/mol_int.py

contdf = pd.DataFrame({'A_atom_id':[s[0] for s in dist],
                                         'A_atom_name':[id2name[s[0]]  for s in dist ],
                                         'A_resname':[id2resname[s[0]]  for s in dist ],
                                         'A_segid':[id2segid[s[0]]  for s in dist ],
                                         'A_resid':[id2resid[s[0]]  for s in dist],
                                         'B_atom_id':[s[1]  for s in dist ],
                                         'B_atom_name':[id2name[s[1]]  for s in dist ],
                                         'B_resname':[id2resname[s[1]]  for s in dist ],
                                         'B_segid':[id2segid[s[1]]  for s in dist ],
                                         'B_resid':[id2resid[s[1]]  for s in dist ],
                                         'dist':[s[2]  for s in dist ],
             })


print(contdf.shape)
contdf.head()

(83860, 11)


,A_atom_id,A_atom_name,A_resname,A_segid,A_resid,B_atom_id,B_atom_name,B_resname,B_segid,B_resid,dist
0,6025,CA,ALA,A,1,6024,N,ALA,A,1,1.488439
1,6026,C,ALA,A,1,6024,N,ALA,A,1,2.478390
2,6027,O,ALA,A,1,6024,N,ALA,A,1,3.301416
3,6028,CB,ALA,A,1,6024,N,ALA,A,1,2.467478
4,6029,N,ARG,A,2,6024,N,ALA,A,1,3.096918


In [81]:
contdf['dist']=pd.to_numeric(contdf['dist'])
contdf = contdf.loc[contdf['A_atom_id'] != contdf['B_atom_id']]
print(contdf.shape)
contdf

(83860, 11)


,A_atom_id,A_atom_name,A_resname,A_segid,A_resid,B_atom_id,B_atom_name,B_resname,B_segid,B_resid,dist
0,6025,CA,ALA,A,1,6024,N,ALA,A,1,1.488439
1,6026,C,ALA,A,1,6024,N,ALA,A,1,2.478390
2,6027,O,ALA,A,1,6024,N,ALA,A,1,3.301416
3,6028,CB,ALA,A,1,6024,N,ALA,A,1,2.467478
4,6029,N,ARG,A,2,6024,N,ALA,A,1,3.096918
...,...,...,...,...,...,...,...,...,...,...,...
83855,13608,CA,LYS,H,122,13616,OXT,LYS,H,122,2.378113
83856,13609,C,LYS,H,122,13616,OXT,LYS,H,122,1.246980
83857,13610,O,LYS,H,122,13616,OXT,LYS,H,122,2.203215
83858,13611,CB,LYS,H,122,13616,OXT,LYS,H,122,3.111546


In [82]:
df_avr = contdf.groupby(['A_resname', 'A_segid','A_resid','B_resname','B_segid','B_resid']).size().reset_index()

df_avr.columns=['A_resname','A_segid','A_resid','B_resname','B_segid'
                                 ,'B_resid','num_']

In [83]:
test = contdf.groupby(['A_resname', 'A_segid','A_resid','B_resname','B_segid','B_resid']).size().reset_index()

In [84]:
test

,A_resname,A_segid,A_resid,B_resname,B_segid,B_resid,0
0,ALA,A,1,ALA,A,1,20
1,ALA,A,1,ARG,A,2,18
2,ALA,A,7,ALA,A,7,20
3,ALA,A,7,ARG,A,8,12
4,ALA,A,7,LYS,A,4,1
...,...,...,...,...,...,...,...
8323,VAL,H,115,THR,H,119,5
8324,VAL,H,115,TYR,G,50,3
8325,VAL,H,115,TYR,H,118,3
8326,VAL,H,115,VAL,G,49,1


In [85]:
pd.DataFrame(test)

,A_resname,A_segid,A_resid,B_resname,B_segid,B_resid,0
0,ALA,A,1,ALA,A,1,20
1,ALA,A,1,ARG,A,2,18
2,ALA,A,7,ALA,A,7,20
3,ALA,A,7,ARG,A,8,12
4,ALA,A,7,LYS,A,4,1
...,...,...,...,...,...,...,...
8323,VAL,H,115,THR,H,119,5
8324,VAL,H,115,TYR,G,50,3
8325,VAL,H,115,TYR,H,118,3
8326,VAL,H,115,VAL,G,49,1


In [45]:
df_avr

,A_resname,A_segid,A_resid,B_resname,B_segid,B_resid,num
0,ALA,A,1,ALA,A,1,20
1,ALA,A,1,ARG,A,2,18
2,ALA,A,7,ALA,A,7,20
3,ALA,A,7,ARG,A,8,12
4,ALA,A,7,LYS,A,4,1
...,...,...,...,...,...,...,...
8323,VAL,H,115,THR,H,119,5
8324,VAL,H,115,TYR,G,50,3
8325,VAL,H,115,TYR,H,118,3
8326,VAL,H,115,VAL,G,49,1


In [26]:
df_avr.info()                                                                          

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8328 entries, 0 to 8327
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   A_resname  8328 non-null   object
 1   A_segid    8328 non-null   object
 2   A_resid    8328 non-null   int64 
 3   B_resname  8328 non-null   object
 4   B_segid    8328 non-null   object
 5   B_resid    8328 non-null   int64 
 6   num_int    8328 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 455.6+ KB


In [151]:
df_avr.to_csv('contacts.csv')

# Пример использования библиотеки pynucl

pdb из базы лежат в '_home/_shared/package_dev/nucdb/pdbfiles'

In [27]:
p=pynucl.nuclstr('1KX5.pdb',format='PDB',auto_detect_entities=True,ref='1KX5_NRF',aln_sel='nlp_0 and (alpha1 or alpha2 or alpha3) and name CA',check_dyad=True)
        

1 frames loaded for None


In [28]:
import json

with open('../_shared/package_dev/nucdb/DB/nlp_containing_structures.json', 'r') as fp:
    nlp_containing_structures = json.load(fp)

In [31]:
nlp_containing_structures['1KX5']['pynucl_parsed']['A']

{'entity': 'histone',
 'type': 'H3',
 'variant': 'canonical_H3',
 'organism': 'Gallus',
 'sequence': 'ARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDLRFQSSAVMALQEASEAYLVALFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA',
 'has_water': True,
 'has_ions': True,
 'has_other': False,
 'side': 1,
 'resid_start': 1,
 'nlp_id': 0,
 'nlp_type': 'octamer'}

In [37]:
pdbid = '1KX5'

In [38]:
        entity_map_dict={}
        for chain,chaindict in nlp_containing_structures[pdbid]['pynucl_parsed'].items():
            if chaindict['entity']=="DNA":
                entity_map_dict[chain]=chaindict['type']
            elif chaindict['entity']=="histone":
                if chaindict['type']!='H1':
                    entity_map_dict[chain]=chaindict['type']+ ('p' if chaindict['side']==1 else 'd')
                else:
                    entity_map_dict[chain]=chaindict['type']
            else:
                entity_map_dict[chain]='Other'
        
        
        contacts_dna=pynucl.a_contacts(p,'protein','protein')

        dm=contacts_dna.get_df()
        dn=dm.groupby(['A_segid','A_resid','A_resname','B_segid','B_resid','B_resname'])['dist'].nunique().reset_index()
        dn.rename({'dist':'num_int'},axis=1,inplace=True)
        dn['A_resname']=dn['A_resname'].map(residues_3to1)
        dn['B_resname']=dn['B_resname'].map(residues_3to1)
        dn['A_entity']=dn['A_segid'].map(entity_map_dict)
        dn['B_entity']=dn['B_segid'].map(entity_map_dict)

In [39]:
dn

,A_segid,A_resid,A_resname,B_segid,B_resid,B_resname,num_int,A_entity,B_entity
0,A,1,A,A,1,A,11,H3p,H3p
1,A,1,A,A,2,R,18,H3p,H3p
2,A,2,R,A,1,A,18,H3p,H3p
3,A,2,R,A,2,R,34,H3p,H3p
4,A,2,R,A,3,T,16,H3p,H3p
...,...,...,...,...,...,...,...,...,...
8323,H,121,A,H,122,K,15,H2Bd,H2Bd
8324,H,122,K,G,6,Q,2,H2Bd,H2Ad
8325,H,122,K,G,20,R,11,H2Bd,H2Ad
8326,H,122,K,H,121,A,15,H2Bd,H2Bd


In [40]:
dn.to_csv('histone_contacts.csv')

# Задачи
1. Отобрать структуры нуклеосом без белков-партнеров:  
    nlp_containing_structures[pdbid]['pynucl_parsed'][chain_name]['entity'] != 'other_protein'
    !! filer out nucleosomes with equal histone variants  
2. Сколько таких структур? из каких они организмов? какие там гистоновые варианты?
3. Обернуть в цикл поиск контактов внутри этих структур между белками, добавлять в таблицу pdbid, 
4. Добавить в таблицу гистоновые варианты
5. [ пока пропустить] c Помощью p.map_obj2ref_resids соотнести resid остатков на структуре с референсом 1KX5